In [26]:
import pandas as pd
import numpy as np
import sklearn as sk

In [27]:
df = pd.read_csv('Data/ListaCobroDetalle2023.csv')
# Ordernar por idCredito
df = df.sort_values(by=['idCredito'])
# Eliminar la fecha de cobro banco
df = df.drop(columns=['fechaCobroBanco'])
# Rellenar los nulos con 0
df = df.fillna(0)
print (df.shape)
df.head()

C:\Users\Felip\AppData\Local\Temp\ipykernel_23064\443114654.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/ListaCobroDetalle2023.csv')


(1681362, 8)


,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,idRespuestaBanco
917168,126519,6,32697369,2,375.47,375.47,0.00,4.0
696242,122158,4635,31989889,72,160.13,160.13,0.00,4.0
182062,109431,4635,30206120,72,160.13,160.13,0.00,04
489450,114565,4635,30872996,72,160.13,160.13,160.13,0
457458,119694,4635,31647299,72,160.13,160.13,0.00,4.0


In [28]:
# Para cada idCredito, restar el consecutivoCobro mínimo de ese grupo para que el orden inicie en 0

# Enumerar en base al índice del valor para cada idCredito
df['orden'] = df.groupby('idCredito').cumcount()
print(df.shape)
print(df.head(10))


(1681362, 9)
        idListaCobro  idCredito  consecutivoCobro  idBanco  montoExigible  \
917168        126519          6          32697369        2         375.47   
696242        122158       4635          31989889       72         160.13   
182062        109431       4635          30206120       72         160.13   
489450        114565       4635          30872996       72         160.13   
457458        119694       4635          31647299       72         160.13   
662737        118214       4635          31447890       72         160.13   
61383         110601       4635          30352104       72         160.13   
233769        111889       4635          30518564       72         160.13   
216863        113139       4635          30674731       72         160.13   
604653        120794       4635          31789209       72         160.13   

        montoCobrar  montoCobrado idRespuestaBanco  orden  
917168       375.47          0.00              4.0      0  
696242       160.13

In [29]:
# Filtra los valores a los 4 mas populares y si no esta dentro de esos 4, lo pone como otro o 1
top4_bancos = df['idBanco'].value_counts().nlargest(4).index
print (top4_bancos) # son 12, 2, 72, 21
# Hacer un diccionario de los 4 bancos mas populares de 0 a 3
# y el resto como 4
bancos = {12: 0, 2: 1, 72: 2, 21: 3}
# Agregamos el resto como 4
bancos.update({x: 4 for x in df['idBanco'].unique() if x not in bancos})
df['idBanco'] = df['idBanco'].apply(lambda x: x if x in top4_bancos else 1)
# Ahora la pasamos a dummy
df = pd.get_dummies(df, columns=['idBanco'], prefix='banco')



Index([2, 12, 14, 72], dtype='int64', name='idBanco')


In [30]:
print (df.head())
print (df.shape)

        idListaCobro  idCredito  consecutivoCobro  montoExigible  montoCobrar  \
917168        126519          6          32697369         375.47       375.47   
696242        122158       4635          31989889         160.13       160.13   
182062        109431       4635          30206120         160.13       160.13   
489450        114565       4635          30872996         160.13       160.13   
457458        119694       4635          31647299         160.13       160.13   

        montoCobrado idRespuestaBanco  orden  banco_1  banco_2  banco_12  \
917168          0.00              4.0      0    False     True     False   
696242          0.00              4.0      0    False    False     False   
182062          0.00               04      1    False    False     False   
489450        160.13                0      2    False    False     False   
457458          0.00              4.0      3    False    False     False   

        banco_14  banco_72  
917168     False     False 

In [31]:
# Para el input del modelo, primero necesitamos agrupar por idCredito y luego el ultimo valor de cada grupo es el output
Cobradores = df.groupby('idCredito')
# Cuenta cuantos cobros se han hecho
print (Cobradores.size())
print (len(Cobradores))
# Ver el maximo y el minimo de cada grupo en tamano
print (Cobradores.size().max())
print (Cobradores.size().min())

idCredito
6           1
4635       12
4914       12
8947       12
9872      402
         ... 
689569      1
689582      1
689593      1
689607      1
689904      4
Length: 41572, dtype: int64
41572
714
1


In [32]:
# Creamos una lista donde cada elemento es el historial de un grupo como lista de listas (filas)
historial_por_grupo = [grupo.values.tolist() for _, grupo in Cobradores]
print (len(historial_por_grupo))
print (historial_por_grupo[0])

41572
[[126519, 6, 32697369, 375.47, 375.47, 0.0, 4.0, 0, False, True, False, False, False]]


In [33]:
for i in range(len(historial_por_grupo)):
    print (len(historial_por_grupo[i]))

1
12
12
12
402
1
1
5
1
1
79
3
1
84
1
3
2
4
1
11
1
84
2
7
4
33
2
10
1
1
11
3
68
4
1
4
23
2
4
4
1
84
1
5
12
9
1
1
4
1
19
4
8
4
4
1
19
11
4
84
15
254
4
1
34
20
4
8
4
12
62
84
1
1
76
2
4
558
10
2
84
8
10
6
10
2
91
12
6
4
3
1
53
1
1
2
6
12
1
39
1
33
77
53
11
1
45
10
1
54
2
1
1
3
1
10
12
2
2
19
1
13
7
2
180
13
14
3
1
1
1
7
1
226
5
4
1
12
2
11
2
2
42
3
12
2
7
8
344
11
7
29
12
190
2
30
1
59
112
121
7
11
6
1
1
12
2
181
1
2
580
2
48
1
39
5
1
10
5
1
2
2
2
7
1
3
1
3
2
25
1
1
3
1
1
2
1
157
362
2
1
3
2
1
2
7
43
4
209
2
3
5
3
24
2
1
5
3
53
1
11
2
12
53
12
5
235
3
1
3
14
1
1
3
2
2
1
1
12
1
1
2
2
12
1
11
49
12
1
1
1
40
35
4
2
2
4
54
1
4
2
2
15
1
1
15
172
1
1
92
1
1
6
20
380
12
2
1
2
2
10
2
4
15
93
116
1
6
1
15
15
13
2
15
13
1
1
1
12
12
1
10
50
2
15
12
2
13
10
13
11
1
473
5
2
2
1
93
83
1
80
1
6
1
12
1
1
83
99
1
215
2
6
12
186
2
2
43
17
12
1
12
89
15
1
2
2
2
1
1
110
64
57
181
1
5
11
1
1
2
1
8
2
4
207
1
1
188
4
1
1
9
6
4
106
367
43
22
11
1
63
9
1
6
7
97
18
6
12
1
1
1
13
1
378
64
367
6
6
24
1
2
1
6
1
12
1


In [22]:
# Calcula el valor de predicción para cada grupo: el último valor de idRespuestaBanco
# 0 = no hubo saldo, 4 = sí se hizo el cobro

predicciones = Cobradores['idRespuestaBanco'].last().reset_index()
predicciones.rename(columns={'idRespuestaBanco': 'valor_prediccion'}, inplace=True)
print(predicciones.head())

   idCredito  valor_prediccion
0       9872               4.0
1      10983               0.0
2      30466               4.0
3      31375               4.0
4      33591               4.0


In [23]:
# Veamos cuantas predicciones tenemos
print (predicciones['valor_prediccion'].value_counts(normalize=True))
# Aplicar un cambio, pasamos de 4 a 1, 26 a 1, y el resto a any a 0
dicccionario = {0: 1, 4: 0, 26: 1}
predicciones['valor_prediccion'] = predicciones['valor_prediccion'].replace(dicccionario)
# Ahora checamos para los que son cualqueir otro valor
predicciones['valor_prediccion'] = np.where(predicciones['valor_prediccion'].isin([0, 1]), predicciones['valor_prediccion'], 0)
# Ahora lo pasamos a 0 y 1
print (predicciones['valor_prediccion'].value_counts(normalize=True))

valor_prediccion
4.0     0.689567
0.0     0.248689
8.0     0.019614
26.0    0.014230
13.0    0.008985
3.0     0.007622
2.0     0.006783
1.0     0.003426
30.0    0.000699
88.0    0.000210
40.0    0.000070
36.0    0.000035
10.0    0.000035
6.0     0.000035
Name: proportion, dtype: float64
valor_prediccion
0.0    0.733655
1.0    0.266345
Name: proportion, dtype: float64


In [24]:
# Ahora visualizemos el inputs y el output del primer grupo
random = np.random.randint(0, len(historial_por_grupo))
# Visualiza el input (X_0:t-1), el último input (X_t) y el output (Y_t) del grupo seleccionado
X_0 = historial_por_grupo[random][:-1]  # Historial sin el último
X_t = historial_por_grupo[random][-1]  # Último input
Y_t = predicciones['valor_prediccion'][random]

# Print the length of the input and output
print("Longitud de X_0:t-1 (historial sin el último):", len(X_0))
print("Longitud de X_t (última fila del grupo):", len(X_0))


Longitud de X_0:t-1 (historial sin el último): 833
Longitud de X_t (última fila del grupo): 833


In [25]:
# Hacer el split de los datos en train, test y validación de {80, 10, 10}
# Primero separamos el input y el output
X = df.drop(columns=['idCredito', 'idRespuestaBanco'])
y = predicciones['valor_prediccion']
# Ahora hacemos el split
X_train, X_temp, y_train, y_temp = X[:int(0.8*len(X))], X[int(0.8*len(X)):], y[:int(0.8*len(X))], y[int(0.8*len(X)):]
X_val, X_test, y_val, y_test = X_temp[:int(0.5*len(X_temp))], X_temp[int(0.5*len(X_temp)):], y_temp[:int(0.5*len(y_temp))], y_temp[int(0.5*len(y_temp)):]
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)
print("Tamaño del conjunto de entrenamiento:", y_train.shape)
print("Tamaño del conjunto de validación:", y_val.shape)
print("Tamaño del conjunto de prueba:", y_test.shape)

Tamaño del conjunto de entrenamiento: (1691337, 11)
Tamaño del conjunto de validación: (211417, 11)
Tamaño del conjunto de prueba: (211418, 11)
Tamaño del conjunto de entrenamiento: (28602,)
Tamaño del conjunto de validación: (0,)
Tamaño del conjunto de prueba: (0,)


In [ ]:
class PreprocesadorCobros:
    def __init__(self, path_csv):
        self.path_csv = path_csv
        self.df = None
        self.Cobradores = None
        self.historial_por_grupo = None
        self.predicciones = None
        self.top4_bancos = None
        self.bancos = None

    # --- Carga y limpieza de datos ---
    def cargar_datos(self):
        self.df = pd.read_csv(self.path_csv)
        return self

    def ordenar_y_limpiar(self):
        self.df = self.df.sort_values(by=['idCredito'])
        if 'fechaCobroBanco' in self.df.columns:
            self.df = self.df.drop(columns=['fechaCobroBanco'])
        self.df = self.df.fillna(0)
        return self

    # --- Procesamiento de columnas ---
    def agregar_orden(self):
        self.df['orden'] = self.df.groupby('idCredito').cumcount()
        return self

    def procesar_bancos(self):
        self.top4_bancos = self.df['idBanco'].value_counts().nlargest(4).index
        self.bancos = {12: 0, 2: 1, 72: 2, 21: 3}
        self.bancos.update({x: 4 for x in self.df['idBanco'].unique() if x not in self.bancos})
        self.df['idBanco'] = self.df['idBanco'].apply(lambda x: x if x in self.top4_bancos else 1)
        self.df = pd.get_dummies(self.df, columns=['idBanco'], prefix='banco')
        return self

    # --- Agrupación y generación de features ---
    def agrupar_por_credito(self):
        self.Cobradores = self.df.groupby('idCredito')
        return self

    def crear_historial_por_grupo(self):
        self.historial_por_grupo = [grupo.values.tolist() for _, grupo in self.Cobradores]
        return self

    # --- Generación de etiquetas ---
    def calcular_predicciones(self):
        self.predicciones = self.Cobradores['idRespuestaBanco'].last().reset_index()
        self.predicciones.rename(columns={'idRespuestaBanco': 'valor_prediccion'}, inplace=True)
        dicccionario = {0: 1, 4: 0, 26: 1}
        self.predicciones['valor_prediccion'] = self.predicciones['valor_prediccion'].replace(dicccionario)
        self.predicciones['valor_prediccion'] = np.where(
            self.predicciones['valor_prediccion'].isin([0, 1]),
            self.predicciones['valor_prediccion'],
            0
        )
        return self

    # --- Ejecutar todo el preprocesamiento ---
    def ejecutar_todo(self):
        self.cargar_datos()
        self.ordenar_y_limpiar()
        self.agregar_orden()
        self.procesar_bancos()
        self.agrupar_por_credito()
        self.crear_historial_por_grupo()
        self.calcular_predicciones()
        return self.split_datos()